<h2><center> Data extraction from the Zomato API to a MySQL database <center/><h2/>

## Extract data from Zomata API

In [1]:
import requests
import pprint as pp
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
def get_city(city, api_key):
    '''
    get_city extracts the Zomato ID and other details for a city 
    which is needed to obtain details on chosen city.
    
    Inputs:
    city: City of interest in string format
    api_key: API key provided by developers.zomato.com
    
    
    Outputs:
    Json object containing a list of cities that meet query
    '''
    
    base_url = 'https://developers.zomato.com/api/v2.1/cities?'
    header = {'user-key': api_key}
    param = {'q': city  }
    
    response = requests.get(base_url,headers=header,params=param)
    
    return response

In [3]:
def get_restaurants(api_key,entity_id,entity_type,sort,query,order,start):
    '''
    get_restaurants extracts a list of restaurants (limit of 100) and their characteristics. Can
    only get 20 results at a time
    
    
    Inputs:
    api_key       : API key provided by developers.zomato.com
    entity_id     : Zomato ID of the restaurant
    entity_type   : Location type (e.g. city, zone, subzone etc)
    sort          : Sort restaurants (e.g. cost, rating)
    query         : City name
    order         : Arrange items in ascending or descinding (asc or desc)
    start         : Fetch results after offset (use an offset of 0 )   
    
    
    Output:
    Json object containing restaurant info    
    '''
    
    base_url = 'https://developers.zomato.com/api/v2.1/search?'
        
    header = {'user-key': api_key}
    param = {'entity_id': entity_id,
             'entity_type': entity_type,
             'q': query,
             'start': start,
             'sort': sort,
             'order': order}
    
    response = requests.get(base_url,headers=header,params=param)
    return response

In [4]:
def Ratings(List_restaurants):
    '''
    Ratings extracts ratings and review information from the restaurants 
    
    Inputs:
    Restaurant_info: container with ratings and review information
    
    Outputs:
    Ratings and review information
    '''
    name, restaurant_id, ratings, ratings_descrip, ratings_votes, reviews  = [], [], [], [], [], []
    
    Restaurant_info = List_restaurants.json()['restaurants']
    res_batch = len(Restaurant_info)

    for j in range(res_batch):

        res_info = List_restaurants.json()['restaurants'][j]['restaurant']
        
        try:
            name.append(res_info['name'])
        except:
            name.append(None)
            
        try:
            restaurant_id.append(int(res_info['id']))
        except:
            restaurant_id.append(None)

        try:
            ratings.append( float( res_info['user_rating']['aggregate_rating'] ) )
        except:
            ratings.append(None)

        try:
            ratings_descrip.append(res_info['user_rating']['rating_text'])
        except:
            ratings_descrip.append(None)

        try:
            ratings_votes.append( float( res_info['user_rating']['votes'] ) )
        except:
            ratings_votes.append(None)
            
        try:
            reviews.append( int( res_info['all_reviews_count'] ) )
        except:
            reviews.append(None)
            
    return name, restaurant_id, ratings, ratings_descrip, ratings_votes, reviews

In [5]:
def Contact(List_restaurants):
    '''
    Contact extracts Contact information on a restaurant
    
    Inputs:
    Restaurant_info: container with ratings and review information
    
    Outputs:
    location information 
    '''   
    address, city, latitude, longitude, zip_code, locality, phone_no = [], [], [], [], [], [], []
    
    Restaurant_info = List_restaurants.json()['restaurants']
    res_batch = len(Restaurant_info)

    for j in range(res_batch):
        res_info = List_restaurants.json()['restaurants'][j]['restaurant']
        
        try:
            address.append( ( res_info['location']['address'] ) )
        except:
            address.append(None)

        try:
            city.append( ( res_info['location']['city'] ) )
        except:
            city.append(None)

        try:
            latitude.append( float( res_info['location']['latitude'] ) )
        except:
            latitude.append(None)

        try:
            longitude.append( float( res_info['location']['longitude'] ) )
        except:
            longitude.append(None)

        try:
            zip_code.append( ( res_info['location']['zipcode'] ) )
        except:
            zip_code.append(None)

        try:
            locality.append( ( res_info['location']['locality'] ) )
        except:
            locality.append(None)

        try:
            phone_no.append( ( res_info['phone_numbers'] ) )
        except:
            phone_no.append(None)
            
    return address, city, latitude, longitude, zip_code, locality, phone_no

In [6]:
def Price(List_restaurants):
    '''
    Price extracts pricing information on a restaurant
    
    Inputs:
    List_restaurants: container with ratings and review information
    
    Outputs:
    Pricing information    
    '''
    price, currency, average_cost_for_two = [], [], []
    
    Restaurant_info = List_restaurants.json()['restaurants']
    res_batch = len(Restaurant_info)
    
    for j in range(res_batch):
        res_info = List_restaurants.json()['restaurants'][j]['restaurant']
        
        try:
            price.append( float( res_info['price_range'] ) )
        except:
            price.append(None)

        try:
            average_cost_for_two.append( float( res_info['average_cost_for_two'] ) )
        except:
            average_cost_for_two.append(None)

        try:
            currency.append( res_info['currency'] )
        except:
            currency.append(None)
        
    return price, currency, average_cost_for_two

In [7]:
def Cuisine(List_restaurants):
    '''
    Cuisine gets details on food and drinks offered by a restaurant
    
    Inputs:
    List_restaurants: container with ratings and review information
    
    Outputs:
    Pricing information  
    '''
    
    no_of_cuisines, cuisine_1, cuisine_2, cuisine_3, cuisine_4 = [], [], [], [], []
    
    
    Restaurant_info = List_restaurants.json()['restaurants']
    res_batch = len(Restaurant_info)
    
    
    for j in range(res_batch):
        res_info = List_restaurants.json()['restaurants'][j]['restaurant']
        
        try:
            no_of_cuisines.append(len((res_info['cuisines']).split(",")))
        except:
            no_of_cuisines.append(None)

        try:                
            cuisine_1.append((res_info['cuisines']).split(",")[0])
        except:
            cuisine_1.append(None)

        try:                
            cuisine_2.append((res_info['cuisines']).split(",")[1])
        except:
            cuisine_2.append(None)

        try:                
            cuisine_3.append((res_info['cuisines']).split(",")[2])
        except:
            cuisine_3.append(None)

        try:                
            cuisine_4.append((res_info['cuisines']).split(",")[3])
        except:
            cuisine_4.append(None)
            
    return no_of_cuisines, cuisine_1, cuisine_2, cuisine_3, cuisine_4       

In [8]:
def Highlights(List_restaurants):
    '''
    Highlights gets details on special features offered by a restaurants
    
    Inputs:
    List_restaurants: container with ratings and review information
    
    Outputs:
    Highlights   
    '''

    no_highlights, vegan, alcohol, gluten_free, takeaway = [], [], [], [], []
    
    Restaurant_info = List_restaurants.json()['restaurants']
    res_batch = len(Restaurant_info)
    
    for j in range(res_batch):
        res_info = List_restaurants.json()['restaurants'][j]['restaurant']
                    
        try:
            no_highlights.append( len(res_info['highlights']) )
        except:
            no_highlights.append(None)

        try:                
            if 'Vegan Options' in res_info['highlights']:
                vegan.append(1)
            else:
                vegan.append(0)
        except:
            vegan.append(None)

        try:                   
            if 'Serves Alcohol' in res_info['highlights']:
                alcohol.append(1)
            else:
                alcohol.append(0)
        except:
            alcohol.append(None)

        try:                    
            if 'Gluten Free Options' in res_info['highlights']:
                gluten_free.append(1)
            else:
                gluten_free.append(0)
        except:
             gluten_free.append(None)

        try:                    
            if 'Takeaway Available' in res_info['highlights']:
                takeaway.append(1)
            else:
                takeaway.append(0)                       
        except:         
            takeaway.append(None)
    
    return no_highlights, vegan, alcohol, gluten_free, takeaway     

In [9]:
def Restaurants(api_key,data_response,entity_type,sort,query,order,no_restaurants,p):
    '''
    This code queries the Zomato database to obtain restaurant information for a given area.
    
    Arguments: (See previous functinos for details)
    no_restaurants: Total number of restaurants to query (Max allowable 100)
    p: city of choice from city query
        
    Output: 
    CSV file with restaurant information for city of choice
    '''
    
    container = data_response.json()['location_suggestions'][p]
    entity_id = container['id']
    country_id = container['country_id']
    country = container['country_name']
    state = container['state_name']
    state_id = container['state_id']
    
    ## Name, ratings
    name, restaurant_id, ratings, ratings_descrip, ratings_votes, reviews = [], [], [], [], [], []
    
    ## Contact details
    address, city, latitude, longitude, zip_code, locality, phone_no = [], [], [], [], [], [], []
    
    ## Pricing information and ordering
    price, currency, average_cost_for_two  = [], [], []
    
    # cuisines
    no_of_cuisines, cuisine_1, cuisine_2, cuisine_3, cuisine_4 = [], [], [], [], []
    
    # highlights
    no_highlights, vegan, alcohol, gluten_free, takeaway = [], [], [], [], []
          
    # Define the vector of starting positions to query api
    start_positions = np.arange(0,no_restaurants,20) 
    
    for i in start_positions:
        
        List_restaurants = get_restaurants(api_key,entity_id,entity_type,sort,query,order,start=i)
            
        name1, restaurant_id1, ratings1, ratings_descrip1, ratings_votes1, reviews1 = Ratings(List_restaurants)
        address1, city1, latitude1, longitude1, zip_code1, locality1, phone_no1 = Contact(List_restaurants)
        price1, currency1, average_cost_for_two1 = Price(List_restaurants)
        no_of_cuisines1, cuisine_11, cuisine_21, cuisine_31, cuisine_41   = Cuisine(List_restaurants)
        no_highlights1, vegan1, alcohol1, gluten_free1, takeaway1  = Highlights(List_restaurants)
        
              
      ##########################################################################################
        name += name1
        restaurant_id += restaurant_id1
        ratings += ratings1
        address += address1
        phone_no += phone_no1
        
        city += city1
        locality += locality1
        
        latitude += latitude1
        longitude += longitude1
        zip_code += zip_code1
        
        price += price1
        currency += currency1
        average_cost_for_two += average_cost_for_two1
        
        reviews += reviews1
        ratings_descrip += ratings_descrip1
        ratings_votes += ratings_votes1
        
        no_highlights += no_highlights1
        vegan += vegan1
        alcohol += alcohol1
        gluten_free += gluten_free1
        takeaway  += takeaway1
        
        no_of_cuisines += no_of_cuisines1
        cuisine_1 += cuisine_11
        cuisine_2 += cuisine_21
        cuisine_3 += cuisine_31
        cuisine_4 += cuisine_41
        
 
    # Define dictionaries  
    dict_1 = {'Name':name, "Restaurant_id": restaurant_id, "Ratings": ratings,'Address': address,'Phone_number': phone_no}
    dict_2 = {'City': city, 'Locality':locality }
    dict_3 = {'Latitude': latitude, 'Longitude':longitude, 'Zip_code': zip_code}
    dict_4 = {'Price':price,  "Currency":currency, "Average_cost_for_two": average_cost_for_two}
    dict_5 = {"No_of_reviews": reviews, 'Ratings_description': ratings_descrip, "Ratings_votes": ratings_votes }
    dict_6 = {"No_of_highlights": no_highlights, "Offers_vegan_options": vegan, "Serves_alcohol": alcohol, "Gluten_free":gluten_free, "Offers_takeaway": takeaway}
    dict_7 = {"No_of_cuisines": no_of_cuisines, 'Cuisine_1': cuisine_1, 'Cuisine_2': cuisine_2, 'Cuisine_3': cuisine_3, 'Cuisine_4': cuisine_4}
    
     # Combine dictionarys
    dict_final =  {**dict_1, **dict_2, **dict_3, **dict_4, **dict_5, **dict_6, **dict_7}     
     
     
    # Save
    df = pd.DataFrame(dict_final)
    
    df['Country']  = country
    df['Country_id'] = country_id
    df['State'] = state
    df['State_id'] = state_id
                                           
    return df

## Create and insert into Mysql Database

In [10]:
import os
import mysql.connector
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as db
from sqlalchemy import create_engine
import pymysql.cursors

%matplotlib inline
!pip install ipython-sql

In [11]:
# Create database  
def Create_db(username,password,country):
    connection = mysql.connector.connect(host = 'localhost',
                                        user = username,
                                        password = password,
                                        use_pure=True)

    cursor = connection.cursor()

    query_table = f"CREATE DATABASE IF NOT EXISTS {country};"

    cursor.execute(query_table)
    connection.commit()
    connection.close()

In [12]:
# Define table structure
def Ratings_Table(connection):
    cursor = connection.cursor()
    
    query_table  =     '''
                CREATE TABLE IF NOT EXISTS Ratings(
                Restaurant_id INT PRIMARY KEY NOT NULL,
                Name TEXT,
                Ratings REAL,
                Ratings_description VARCHAR(20),
                Ratings_votes REAL,
                No_of_Reviews INT);
                ''' 
    cursor.execute(query_table)
      
    connection.commit()
    
    return connection

def Price_Table(connection):
    cursor = connection.cursor()
    
    query_table =     '''
                    CREATE TABLE IF NOT EXISTS Price(
                    Restaurant_id INT PRIMARY KEY NOT NULL,
                    Price REAL,
                    Average_cost_for_two REAL,
                    Currency VARCHAR(5));
                    '''
        
    cursor.execute(query_table)
    connection.commit()
    
    return connection

def Location_Table(connection):
    cursor = connection.cursor()
    
    query_table =   '''
                    CREATE TABLE IF NOT EXISTS Location(
                    Restaurant_id INT PRIMARY KEY NOT NULL,
                    Address TEXT,
                    Zip_code TEXT,
                    Phone_number TEXT,
                    Locality TEXT,
                    State TEXT,
                    State_id INT,
                    Country TEXT NOT NULL,
                    Country_id INT NOT NULL,
                    Latitude REAL,
                    Longitude REAL);
                    '''
    cursor.execute(query_table)
    connection.commit()
    
    return connection


def Cuisine_Table(connection):
    cursor = connection.cursor()
    
    query_table =    '''
                    CREATE TABLE IF NOT EXISTS Cuisine(
                    Restaurant_id INT PRIMARY KEY NOT NULL,
                    No_of_cuisines INT,
                    Cuisine_1 VARCHAR(40),
                    Cuisine_2 VARCHAR(40),
                    Cuisine_3 VARCHAR(40),
                    Cuisine_4 VARCHAR(40),
                    Vegan_options INT,
                    Serves_alcohol INT,
                    Gluten_free INT,
                    Offers_takeaway INT);
                    '''
        
    cursor.execute(query_table)
    connection.commit()
    
    return connection

In [13]:
# Create tables
def Create_tables(username,password,database):
        
    '''
    Creates_tables takes as inputs the username and password for the MYSQL instance and the database of choice. 
    It then creates tables based on a linked_list structure
    Default tables: Ratings, Price, Location and Cuisine
    '''
    connection = mysql.connector.connect(host = 'localhost',
                                        user = username,
                                        password = password,
                                        database=database,
                                        charset = 'utf8',
                                        use_pure=True)
    cursor = connection.cursor()

    connection = Ratings_Table(connection)
    connection = Price_Table(connection)
    connection = Location_Table(connection)
    connection = Cuisine_Table(connection)
    

    connection.commit()
    connection.close()


In [14]:
def Insert_tables(username,password,database,df):
    
    '''
    Insert_tables takes as inputs the username and password for the MYSQL instance,
    the database of choice for insertion and imputes data.
    The default databases are; Ratings, Price, Location and Cuisine  
    '''

    connection = mysql.connector.connect(host = 'localhost',
                                        user = username,
                                        password = password,
                                        database=database,
                                         charset = 'utf8',
                                        use_pure=True)

    cursor = connection.cursor()

    Ratings_val = df[['Restaurant_id', 'Name','Ratings','Ratings_description','Ratings_votes', 'No_of_reviews']]
    Ratings_query = "INSERT INTO Ratings VALUES(%s,%s,%s,%s,%s,%s);"
    Ratings_list = list(Ratings_val.to_records(index=False).tolist())
    cursor.executemany(Ratings_query, Ratings_list)
    connection.commit()

    Price_val = df[['Restaurant_id', 'Price','Average_cost_for_two','Currency']]
    Price_query = "INSERT INTO Price VALUES(%s,%s,%s,%s);"
    Price_list = list(Price_val.to_records(index=False).tolist())
    cursor.executemany(Price_query, Price_list)
    connection.commit()
   

    Location_val = df[['Restaurant_id', 'Address','Zip_code', 'Phone_number',
                      'Locality','State', 'State_id','Country','Country_id','Latitude', 'Longitude']]
    Location_query = "INSERT INTO Location VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    Location_list = list(Location_val.to_records(index=False).tolist())
    cursor.executemany(Location_query, Location_list)
    connection.commit()


    Cuisine_val = df[['Restaurant_id', 'No_of_cuisines','Cuisine_1','Cuisine_2','Cuisine_3', 'Cuisine_4',
                     'Offers_vegan_options','Serves_alcohol','Gluten_free','Offers_takeaway']]
    Cuisine_query = "INSERT INTO Cuisine VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    Cuisine_list = list(Cuisine_val.to_records(index=False).tolist())
    cursor.executemany(Cuisine_query, Cuisine_list)   
    connection.commit()
    
    
    connection.close()
    

In [ ]:
city = "Edmonton"
country = "Canada"
api_key = input("Input your Zomato API key: ")

In [16]:
data_response = get_city(city,api_key)
pp.pprint(data_response.json(),indent=3)

{  'has_more': 0,
   'has_total': 0,
   'location_suggestions': [  {  'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_37.png',
                                 'country_id': 37,
                                 'country_name': 'Canada',
                                 'discovery_enabled': 0,
                                 'has_go_out_tab': 0,
                                 'has_new_ad_format': 0,
                                 'id': 334,
                                 'is_state': 0,
                                 'name': 'Edmonton, AB',
                                 'should_experiment_with': 0,
                                 'state_code': 'AB',
                                 'state_id': 119,
                                 'state_name': 'Alberta'},
                              {  'country_flag_url': 'https://b.zmtcdn.com/images/countries/flags/country_216.png',
                                 'country_id': 216,
                             

In [17]:
entity_id=334
query='Edmonton'
entity_type='city'
sort='rating'   
no_restaurants=100
p = 0
order = 'desc'

df = Restaurants(api_key,data_response,entity_type,sort,query,order,no_restaurants,p)

In [18]:
df.head()

,Name,Restaurant_id,Ratings,Address,Phone_number,City,Locality,Latitude,Longitude,Zip_code,...,Offers_takeaway,No_of_cuisines,Cuisine_1,Cuisine_2,Cuisine_3,Cuisine_4,Country,Country_id,State,State_id
0,Duchess Bake Shop,16650433,4.9,10718 124 Street NW T5M 0H1,+1 (780) 488-4999,Edmonton,Westmount,53.551394,-113.536163,T5M 0H1,...,1,2,Bakery,Desserts,None,None,Canada,37,Alberta,119
1,Pho Hoan Pasteur,16649257,4.9,"11443 Kingsway Avenue, Edmonton T5G3E8",+1 (780) 761-1989,Edmonton,Kingsway,53.564936,-113.518684,T5G3E8,...,1,1,Vietnamese,None,None,None,Canada,37,Alberta,119
2,Boualouang Laos and Thai Cuisine,16649275,4.8,"10569 97 St NW, Edmonton T5H",+1 (780) 423-4207,Edmonton,Chinatown & McCauley,53.549801,-113.488370,T5H,...,1,1,Thai,None,None,None,Canada,37,Alberta,119
3,Lan's Asian Grill,16649225,4.8,"11828 103 Street Northwest, Edmonton T5G 2J2",+1 (780) 478-8805,Edmonton,Kingsway,53.570841,-113.497619,T5G 2J2,...,1,3,Asian,Thai,Vietnamese,None,Canada,37,Alberta,119
4,Cafe Amore Bistro,16650829,4.7,"10807 106 Ave Northwest, Edmonton T5H4A7",+1 (780) 477-7896,Edmonton,Central McDougall,53.549688,-113.506893,T5H4A7,...,1,2,Italian,Seafood,None,None,Canada,37,Alberta,119


In [ ]:
username = input("Input username for database connection: ")
password = input("Input your password for database connection: ")

Create_db(username,password,country)
Create_tables(username,password,country)
Insert_tables(username,password,country,df)

## Query database

### Simple queries

In [20]:
connection = mysql.connector.connect(host = 'localhost',
                                        user = username,
                                        password = password,
                                        database=country,
                                         charset = 'utf8',
                                        use_pure=True)

cursor = connection.cursor()

sql_query = "SELECT * FROM Ratings ORDER BY Restaurant_id DESC LIMIT 10;"

cursor.execute(sql_query)

results = cursor.fetchall()

for result in results:
    print(result)

(18259298, 'Seoul Fried Chicken', 4.2, 'Very Good', 144.0, 77)
(16651724, 'Farrow Sandwiches', 4.3, 'Very Good', 190.0, 52)
(16651666, 'Bar Bricco', 4.4, 'Very Good', 183.0, 44)
(16651589, 'Café Bicyclette', 4.3, 'Very Good', 198.0, 69)
(16651525, 'Rge Rd', 4.6, 'Excellent', 356.0, 111)
(16651453, 'Meat', 4.5, 'Excellent', 541.0, 204)
(16651379, 'Woodwork', 4.3, 'Very Good', 400.0, 149)
(16651370, 'Japonais Bistro', 4.4, 'Very Good', 482.0, 163)
(16651342, 'Canteen', 4.2, 'Very Good', 400.0, 148)
(16651322, 'De Dutch', 4.2, 'Very Good', 385.0, 126)


In [21]:
sql_query2 = '''
            SELECT * FROM Ratings 
            JOIN Location 
            USING (Restaurant_id)
            ORDER BY Restaurant_id DESC
            LIMIT 2;
            '''
cursor.execute(sql_query2)

results = cursor.fetchall()

for result in results:
    print(result)
    print("\n")

(18259298, 'Seoul Fried Chicken', 4.2, 'Very Good', 144.0, 77, '7904 104 Street NW, Edmonton T6E 1V1', 'T6E 1V1', '+1 (780) 761-3616', 'University & Old Strathcona', 'Alberta', 119, 'Canada', 37, 53.51545, -113.498095)


(16651724, 'Farrow Sandwiches', 4.3, 'Very Good', 190.0, 52, '8422 109 Street NW, Edmonton T6G', 'T6G', '+1 (780) 757-4160', 'University & Old Strathcona', 'Alberta', 119, 'Canada', 37, 53.5231228, -113.5120398)




In [22]:
sql_query3 = '''
            SELECT price, AVG(Average_cost_for_two) as couple_price FROM Price
            GROUP BY price
            ORDER BY price;
            '''
cursor.execute(sql_query3)

results = cursor.fetchall()

for result in results:
    print(result)

(1.0, 9.6875)
(2.0, 25.0)
(3.0, 39.83870967741935)
(4.0, 72.0)
